<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/openai_api/06-gpt-4v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
from openai import OpenAI

# client = OpenAI()
# OpenAI Python SDK v1.0 更新后的使用方式
# 使用代理方式，需要修改base_url
client = OpenAI(
    api_key="XXX", # 你的KEY
    base_url="https://vip.apiyi.com/v1"
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木质栈道穿过茂密的绿色草甸。栈道直直地向前延伸，并在前方消失于视线之外。周围的自然风景非常宁静和美丽，天空呈现出澄清的蓝色，间杂着几朵白云。草地上不同深浅的绿色交织在一起，远处可以看到一些树木和灌木丛。整体画面给人一种平和与自然和谐相处的感觉。这是一个很好的地方进行散步或者观察自然。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [ ]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [2]:
def query_image_description(url, prompt="介绍下这幅图?"):
    # client = OpenAI()  # 初始化 OpenAI 客户端

    client = OpenAI(
    api_key="XXX", # 你的KEY
    base_url="https://vip.apiyi.com/v1"
    )

    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )

    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [ ]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的比较图，展示了两种不同状态的狗，通过夸张的方式表达人们在不同工作强度下可能出现的变化。左边的图显示了一只具有强壮肌肉的、类似人类健身者体型的狗，上方的文字标注为“16岁的我，工作后的我”，暗示这是年轻时的强健状态。右边的图显示了一只看起来较为普通、稍显憔悴和困倦的狗，这代表了工作后体力与精力的消耗。

每只狗下面都有一些描述文字，如左边的狗有描述“我觉得一光光鲜明、未来可期、人世鲜姿势不够，精神疲劳身体受罪”，暗示了虽然外表强壮但内心感受到疲惫和劳累。右边的狗描述了“好


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [5]:
from openai import OpenAI
import base64
import requests
import json

# client = OpenAI()  # 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX", # 你的KEY
    base_url="https://vip.apiyi.com/v1"
)
def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://vip.apiyi.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/main/openai_api/images/gdp_1980_2020.jpg?raw=1)

In [6]:
!mkdir -p images  # Creates the 'images' directory if it doesn't exist
# 【20250105】 Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp']
!wget https://raw.githubusercontent.com/AlexFly666/002-openai-quickstart-jike-peng/main/openai_api/images/gdp_1980_2020.png -O ./images/gdp_1980_2020.png
content = query_base64_image_description("./images/gdp_1980_2020.png")
print(content)

--2025-01-05 13:31:26--  https://raw.githubusercontent.com/AlexFly666/002-openai-quickstart-jike-peng/main/openai_api/images/gdp_1980_2020.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196664 (192K) [image/png]
Saving to: ‘./images/gdp_1980_2020.png’

./images/gdp_1980_2 100%[===================>] 192.05K  --.-KB/s    in 0.03s   

2025-01-05 13:31:26 (5.49 MB/s) - ‘./images/gdp_1980_2020.png’ saved [196664/196664]

这幅图表显示了1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。数据以万亿美元计算，并用不同颜色的线条表示每个国家的GDP走势，其中蓝色代表美国，红色代表中国，紫色代表日本，绿色代表德国。

从图表中可以看出以下几点：
1. 美国的GDP在这四十年间持续增长，且总体呈上升趋势，从1980年的约3万亿美元增长到2020年的超过20万亿美元。
2. 中国的GDP增长最为显著，从1980年的不到1万亿美元急剧上升到2020年的接近15万亿美元。中国的经济增长尤其在2000年后加速。
3. 日本的GDP在1980年到1995年间增长较快，之后增长放缓，并在1995年到2015年间基本保持稳定。2020年的GDP略有下降。
4. 德国的GDP增长相对平稳，从19

#### 使用 GPT-4V 识别手写体笔记

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/main/openai_api/images/handwriting_0.jpg?raw=1)

In [ ]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/main/openai_api/images/handwriting_1.jpg?raw=1)

In [ ]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework:


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。